# Preprocessing .mat for NWB

In [42]:
import numpy as np
import h5py
import os
import yaml
import pandas as pd
from datetime import datetime
import shutil
import importlib
import numpy as np
import pandas as pd
import convert_to_nwb_for_PL
import converters.Initiation_nwb
import scipy.io
import warnings


# Load data 

## .mat files

In [ ]:
csv_file = "data/Subject_Session_Selection.csv"
PL = "data/mouse/General"
PLALL = "data/mouse/Recording"
PLALL_number1 = "data/mouse/Recording/PL200_D1.mat"
output_folder = "data"

In [ ]:
PL200 = [os.path.join(PL, f) for f in os.listdir(PL)][0]
PL200


In [ ]:
for file_name in sorted(os.listdir(PLALL)):
    i += 1
    file_path = os.path.join(PLALL, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.mat'):
        print(f"Processing file: {file_name}")

### General

In [ ]:
from scipy.io import loadmat
General_data = loadmat(PL200)

In [ ]:
type(General_data)

In [ ]:
print(General_data.keys())


In [ ]:
print(General_data["__header__"])


In [ ]:
print(General_data["__version__"])


In [ ]:
print(General_data["__globals__"])


In [ ]:

print(General_data["LFP_Data"])



In [ ]:
# Mouse Name
print(General_data["LFP_Data"][0][0][0][0][0][0])


In [ ]:
# id session 
print(General_data["LFP_Data"][0][0][4])
unique_values, unique_indices = np.unique(General_data["LFP_Data"][0][0][4], return_index=True)
print("Unique values:", unique_values)
print("Indices:", unique_indices)



In [ ]:
i=0
# Start date (dd.mm.yy)
print(General_data["LFP_Data"][0][0][5][unique_indices[1]])

# Start time (hhmmss)
## hh
hh = str(General_data["LFP_Data"][0][0][5][unique_indices[i]][3][0][0])
## mm
if len(str(General_data["LFP_Data"][0][0][5][unique_indices[i]][4][0][0])) == 1:
    mm= "0" + str(General_data["LFP_Data"][0][0][5][unique_indices[i]][4][0][0])
else:
    mm = str(General_data["LFP_Data"][0][0][5][unique_indices[i]][4][0][0])
## ss
ss = str(General_data["LFP_Data"][0][0][5][unique_indices[i]][5][0][0])

start_time = hh + mm +  ss[:2]
print("Start time:", start_time)


#Behavior Type
print(str(General_data["LFP_Data"][0][0][6][unique_indices[i]][0][0]))


In [ ]:
# Strain
print(General_data["LFP_Data"][0][0][1][0][0][0])

In [ ]:
# sex 
print(General_data["LFP_Data"][0][0][2][0][0][0])

In [ ]:
# birth date 
print(str(General_data["LFP_Data"][0][0][3][0][0][0][0]))

In [ ]:
# trial onset 
rien = unique_indices[1]
trial = np.asarray(General_data["LFP_Data"][0][0][8]).flatten()
trialonset_avant= trial[0:rien]

In [ ]:

print(General_data["LFP_Data_description"])

### ALL (number1)

In [ ]:
from scipy.io import loadmat
pli = loadmat(PLALL_number1)

In [ ]:
pli.keys()

In [ ]:
# Stim_times
stim_onset= np.asarray(pli["Stim_times"][0][0][1][0])/1000
# Catch_times
catch_onset = np.asarray(pli["Catch_times"][0][0][0][0])/1000
# trial_onset
all_onsets = np.concatenate([stim_onset, catch_onset])
all_onsets_sorted = np.sort(all_onsets)
print(all_onsets_sorted)
len(all_onsets_sorted)



In [ ]:
Valve_times= np.asarray(pli["Valve_times"][0][0][1][0])/1000
Valve_times

In [ ]:
# Compare trialonset_avant and all_onsets_sorted
trialonset_avant_sorted = np.sort(trialonset_avant)
are_equal = np.array_equal(trialonset_avant_sorted, all_onsets_sorted)
print("Are trialonset_avant and all_onsets_sorted equal?", are_equal)

# Show differences if not equal
if not are_equal:
    diff_in_avant = np.setdiff1d(trialonset_avant_sorted, all_onsets_sorted)
    diff_in_all = np.setdiff1d(all_onsets_sorted, trialonset_avant_sorted)
    print("In trialonset_avant but not in all_onsets_sorted:", diff_in_avant)
    print("In all_onsets_sorted but not in trialonset_avant:", diff_in_all)

# CSV file


In [1]:
import pandas as pd 
csv_data = pd.read_csv("data/Subject_Session_Selection.csv", sep=";")
csv_data.columns = csv_data.columns.str.strip() 
csv_data.columns

Index(['Mouse Name', 'User (user_userName)', 'Ear tag',
       'Start date (dd.mm.yy)', 'End date', 'Sex_bin', 'strain', 'mutations',
       'Birth date', 'licence', 'DG', 'ExpEnd', 'Created on', 'Session',
       'Session Date (yyymmdd)', 'Start Time (hhmmss)', 'Behavior Type',
       'Session Type', 'Mouse Age (d)', 'Weight of Reference',
       'Weight Session', 'Trial_onset', 'stim_onset', 'catch_onset',
       'Responses_times', 'EMG', 'PtA', 'dCA1', 'mPFC', 'wM1', 'wS1', 'wS2',
       'antM1'],
      dtype='object')

In [2]:
csv_data

,Mouse Name,User (user_userName),Ear tag,Start date (dd.mm.yy),End date,Sex_bin,strain,mutations,Birth date,licence,...,catch_onset,Responses_times,EMG,PtA,dCA1,mPFC,wM1,wS1,wS2,antM1
0,PL200,Pierre Le Merre,Unknown,19.06.2014,19.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,47.643;72.935;121.426;145.417;163.214;184.412;...,417.655;478.939;590.514;656.5;791.869;817.26;8...,1.4367e-05;4.53e-05;-4.7600000000000005e-05;-1...,2.266e-06;-4.6933999999999995e-05;2.5e-05;1.44...,0.00011766600000000001;0.000107534;0.0001252;0...,0.000100134;8.0066e-05;9.813399999999999e-05;8...,0.000147734;5.24e-05;0.00010920000000000001;0....,4.2134e-05;2.9134e-05;3.4734e-05;3.4734e-05;4....,NaN,NaN
1,PL200,Pierre Le Merre,Unknown,20.06.2014,20.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,71.961;423.095;545.167;665.838;700.83;725.726;...,20.675;43.669;86.258;241.935;283.728;453.086;6...,1.1167e-05;6.433e-06;-1.3667e-05;7.4e-06;3.26e...,-2.5866e-05;-4.0866e-05;-2.3265999999999997e-0...,-0.000133534;-0.000114866;-0.000118134;-9.7866...,-5.8e-05;-3.52e-05;-2.5e-05;-2.0734e-05;-3.613...,-1.4934e-05;-3.26e-05;-6.7534e-05;-7.2934e-05;...,-3e-06;2.2e-06;-1.066e-06;2.334e-06;5.34e-07;5...,NaN,NaN
2,PL200,Pierre Le Merre,Unknown,21.06.2014,21.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,18.805;50.293;63.69;111.077;143.271;166.266;37...,85.682;183.86;228.649;269.84;288.737;446.299;4...,-1.8433e-05;5.6067e-05;4.97e-05;4.5267e-05;2.3...,3.4866e-05;5.4134e-05;6.9266e-05;8.2134e-05;7....,-0.00023266600000000003;-0.000221666;-0.000179...,9.313399999999999e-05;7.64e-05;0.0001322000000...,0.000129334;0.000124934;0.000165866;0.0001132;...,4.9133999999999994e-05;6.68e-05;0.000101734;3....,NaN,NaN
3,PL200,Pierre Le Merre,Unknown,22.06.2014,22.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,76.43;145.711;342.966;367.161;384.355;404.851;...,100.622;128.414;163.308;181.603;242.889;271.58...,-1.6133000000000003e-05;-2.1267e-05;-1.967e-06...,-1.52e-05;-2.86e-05;-2.9266e-05;-3.5066e-05;-3...,1.4934e-05;1.1065999999999999e-05;2.9934e-05;6...,-1.34e-07;6.0934e-05;4.46e-05;4.7066e-05;7.08e...,2.734e-06;-1.46e-05;-1.6934e-05;-5.86660000000...,-0.000130734;-9.76e-05;-9.153400000000001e-05;...,NaN,NaN
4,PL200,Pierre Le Merre,Unknown,23.06.2014,23.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,46.413;72.107;112.399;126.394;149.689;173.383;...,22.119;98.2;237.266;261.563;276.756;328.048;34...,5.9e-06;1.5367e-05;7.433000000000001e-06;-1.39...,-5.1200000000000004e-05;-5.22e-05;-4.6134e-05;...,0.0002002;0.000186734;0.000180934;0.0002038000...,-8.4666e-05;-9.073399999999999e-05;-8.46e-05;-...,-8.6534e-05;-9.7e-05;-9.7334e-05;-5.4733999999...,1.54e-05;1.666e-06;1.8e-06;9.066e-06;1.6134e-0...,NaN,NaN
5,PL200,Pierre Le Merre,Unknown,24.06.2014,24.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,24.694;48.381;128.462;247.737;309.321;340.115;...,85.873;102.968;144.66;168.352;183.149;209.244;...,2.9166999999999997e-05;-3.3367e-05;-2.1767e-05...,0.000119066;0.0001192;6.46e-05;9.8334000000000...,-1.1e-05;-1.0933999999999999e-05;-5.1734e-05;-...,-0.0001172;-0.00017353399999999998;-0.00014113...,2e-06;-4.9133999999999994e-05;-0.0001676;-8.2e...,4.4934e-05;3.2666e-05;6.934e-06;2.526600000000...,NaN,NaN
6,PL200,Pierre Le Merre,Unknown,25.06.2014,25.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,23.452;85.835;142.223;157.719;179.812;200.808;...,47.642;69.037;101.633;123.326;257.195;279.888;...,1.3667e-05;1.6329999999999999e-06;2.1e-06;1.80...,-8.04e-05;-7.9334e-05;-4.7466e-05;-7.8734e-05;...,0.00020546600000000001;0.000214266;0.000237066...,-7.4066e-05;-5.1866e-05;-6.346599999999999e-05...,-0.0001292;-8.400000000000001e-05;-6.72e-05;-9...,-2.0266e-05;-2.8266e-05;-1.74e-05;-4.533399999...,NaN,NaN
7,PL200,Pierre Le Merre,Unknown,26.06.2014,26.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,59.725;91.014;105.813;124.606;148.003;169.398;...,187.393;235.18;260.173;300.464;317.661;350.255...,1.24e-05;1.9767e-05;-3e-07;-1.0167000000000001...,-6.379999999999999e-05;-9.8466e-05;-0.0001152;...,0.00021840000000000002;0.0002174

# TOTAL CSV

In [ ]:
importlib.reload(converters.Initiation_nwb)
output_path = converters.Initiation_nwb.files_to_csv(PL=PL, PLALL=PLALL, csv_file=csv_file)

 # TOTAL _-_-_-_-_-_-_-_
 

In [ ]:

importlib.reload(convert_to_nwb_for_PL)
nwb_path = convert_to_nwb_for_PL.convert_data_to_nwb_pl(csv_file=csv_file, output_folder="data", mouses_name = ["PL200"])